**Условие: используйте источник rate, напишите код, который создаст дополнительный столбец, который будет выводить сумму только нечётных чисел.**

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=76078ed65de92c02cf818b8fad22043f7c0ceaba1ed7eccbd769c645f1a92c02
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum
from pprint import pprint

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Создаем SparkSession
spark = SparkSession.builder.appName("sum_of_odd_numbers").getOrCreate()
# Создаем поток данных с использованием источника данных `rate`
streaming_df = spark \
.readStream \
.format("rate") \
.option("rowsPerSecond", 1) \
.load()
# Оставляем только те строки, где значение нечетное
filtered_df = streaming_df.filter(col("value") % 2 == 1)
# Суммирование
sum_odds = filtered_df.agg(sum("value"))
# Выводим результат
query = sum_odds \
.writeStream \
.outputMode("complete") \
.format("console") \
.start()
query.awaitTermination(5)
query.stop()